### Code to visualise training and performance of Learn Transform.ipynb

#### Imports and parameter definitions

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

import torch

import autoencoders
import learning_plotters as lp
import Double_Pendulum.robot_parameters as robot_parameters

import matplotlib
matplotlib.rcParams['font.family']   = 'serif'
matplotlib.rcParams['font.serif']    = ['Times New Roman']
matplotlib.rcParams['mathtext.fontset'] = 'dejavuserif'

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

rp = robot_parameters.LUMPED_PARAMETERS.copy()
rp["m0"] = 0.0

In [ ]:
save_folder = "Models/New_try"
os.makedirs(save_folder, exist_ok=True)
file_counter = 0
train_clockwise = False

#### Load models

In [ ]:
dir_name = "NN_202505141642(half-q)"

dir_path = os.path.join(os.getcwd(), "Models", dir_name)
nn_filename = "NN_202505141642" + "_0.pth"
nn_filepath = os.path.join(dir_path, nn_filename)

model_path = nn_filepath
model = autoencoders.Autoencoder_double(rp).to(device)  # Initialize model architecture
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))  # Load weights

model_ana = autoencoders.Analytic_transformer(rp)

models = [model_ana, model]
model_names = ["Analytic", "Learned"]

In [ ]:
def load_loss(load_loss_path):
	losses_dict = torch.load(load_loss_path, map_location = "cpu", weights_only=True)
	return losses_dict["train"], losses_dict["val"]

#### Visualize loss plot, single model

In [ ]:
single_losses_path = "Models/NN_202505221832/losses.pt"
train_loss, val_loss = load_loss(single_losses_path)

In [ ]:
lp.plot_loss(train_loss, val_loss, log = True, save_folder = save_folder)

#### Plot loss for multiple models

In [ ]:
loss_paths = [
    "Models/NN_202505221832/losses.pt",
    "Models/NN_202505221839/losses.pt",
    "Models/NN_202505221919/losses.pt"
]

In [ ]:
train_losses = []
val_losses   = []

for loss_path in loss_paths:
	train_loss, val_loss = load_loss(loss_path)
	train_losses.append(train_loss)
	val_losses.append(val_loss)

In [ ]:
lp.plot_losses_vs_epoch(train_losses, val_losses, save_folder = save_folder)

#### Plot Yin-Yang for $\theta$ vs $x, y$

In [ ]:
# Define the number of grid points along each dimension.
n_points = 200

lp.plot_yinyang(n_points, save_folder, file_counter, train_clockwise, models, model_names, rp, device) #TODO: Fill in